In [ ]:
import math
import logging
import os
import glob
import numpy as np
import pandas as pd
import scipy.signal as sig
import altair as alt

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

from spinorama.load import graph_melt, load_normalize
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.graph import graph_spinorama, graph_freq
from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_build, peq_freq, peq_graph_measurements

In [ ]:
nb_points = 1000
flat_u = pd.DataFrame(
    {
        "Freq": np.logspace(1.0 + math.log10(2), 4.0 + math.log10(2), nb_points),
        "On Axis": [0] * nb_points,
    }
)
flat = graph_melt(flat_u)

In [ ]:
my_fs = 48000
eq_dirs = glob.glob("../datas/eq/*")
peqs = {}
for d in eq_dirs:
    if os.path.isdir(d):
        speaker = os.path.basename(d)
        iirname = "{0}/iir.txt".format(d)
        if os.path.isfile(iirname):
            peqs[d] = parse_eq_iir_rews(iirname, my_fs)
print("Found {0} IIR eq".format(len(peqs)))

In [ ]:
freq = flat_u["Freq"].to_numpy()
dff = []
dff.append(pd.DataFrame({"Freq": freq}))
eq_graphs = {}
for speaker, peq in peqs.items():
    req = peq_build(freq, peq)
    dff.append(pd.DataFrame({speaker: req}))
    g_params = {
        "xmin": 20,
        "xmax": 20000,
        "ymin": -10,
        "ymax": 10,
        "width": 400,
        "height": 250,
    }
    eq_graphs[speaker] = graph_freq(
        graph_melt(pd.DataFrame({"Freq": freq, speaker: req})), g_params
    )

eq_all = pd.concat(dff, axis=1)

In [ ]:
g_params = {
    "xmin": 20,
    "xmax": 20000,
    "ymin": -100,
    "ymax": 100,
    "width": 1200,
    "height": 750,
}
graph_freq(graph_melt(eq_all), g_params)

In [ ]:
gg = alt.vconcat()
for t, g in eq_graphs.items():
    title = os.path.basename(t)
    gg &= g.properties(title=title)

In [ ]:
gg